In [1]:
import pandas as pd
df_rating = pd.read_csv('parsed_json/face-rating-data-2.csv', index_col = None)
df_guess_who_temp1 = pd.read_csv('parsed_json/guess-who-data-1.csv', index_col = None)
df_guess_who_temp2 = pd.read_csv('parsed_json/guess-who-data-2.csv', index_col = None)
df_guess_who1 = df_guess_who_temp1.dropna(subset=['question', 'eliminatedFaces'])
df_guess_who2 = df_guess_who_temp2.dropna(subset=['question', 'eliminatedFaces'])

In [2]:
print(df_guess_who1.info())
print(df_guess_who2.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3130 entries, 3 to 4135
Data columns (total 5 columns):
allFaces            3130 non-null object
eliminatedFaces     3130 non-null object
question            3130 non-null object
questionResponse    3130 non-null object
uniqueID            3130 non-null int64
dtypes: int64(1), object(4)
memory usage: 146.7+ KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 16014 entries, 0 to 21107
Data columns (total 5 columns):
allFaces            16014 non-null object
eliminatedFaces     16014 non-null object
question            16014 non-null object
questionResponse    16014 non-null object
uniqueID            16014 non-null int64
dtypes: int64(1), object(4)
memory usage: 750.7+ KB
None


In [8]:
temp = pd.merge(df_guess_who1, df_guess_who2, how='outer', indicator='indicator_column') 
outfile = open('temp-data.csv','w')
temp.to_csv(outfile, index = False)
outfile.close() 
temp = pd.read_csv("temp-data.csv")
temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19144 entries, 0 to 19143
Data columns (total 6 columns):
allFaces            19144 non-null object
eliminatedFaces     19144 non-null object
question            19144 non-null object
questionResponse    19144 non-null object
uniqueID            19144 non-null int64
indicator_column    19144 non-null object
dtypes: int64(1), object(5)
memory usage: 897.5+ KB


In [3]:
all_rows = []

unique_by_faceID = df_rating.drop_duplicates(subset=['faceID'], keep='first')
for qIndex, q in temp.iterrows():
    allFaces = q['allFaces'].split(',')
    for faceID in allFaces:
        eliminated = faceID in q['eliminatedFaces']
        faces = unique_by_faceID[unique_by_faceID['faceID'] == faceID]
        all_rows.append([
            faceID,
            q['question'],
            q['questionResponse'],
            eliminated,
            faces.iloc[0]['ethnicity'],
            faces.iloc[0]['eye'],
            faces.iloc[0]['gender'],
            faces.iloc[0]['hair'],
            q['allFaces'],
            q['eliminatedFaces']
        ])


In [5]:
import re 
import csv
csv_file = open("parsed_json/temp-data.csv", "w")

# columns = ['faceId', 'age', 'ethnicity', 'gender', 'hair', 'questions', 'responses']
csv_writer = csv.writer(csv_file)
# csv_writer.writeheader()
csv_writer.writerow(['faceId', 'question', 'response', 'eliminated?',
                     'ethnicity', 'eyes', 'gender', 'hair', 'allFaces', 
                     'eliminatedFaces'])
csv_writer.writerows(all_rows)

csv_file.close()

In [6]:
data = pd.read_csv("parsed_json/temp-data.csv")

In [ ]:
import numpy as np

In [96]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306304 entries, 0 to 306303
Data columns (total 10 columns):
faceId             306304 non-null object
question           306304 non-null object
response           306304 non-null object
eliminated?        306304 non-null bool
ethnicity          297673 non-null object
eyes               306304 non-null object
gender             306304 non-null object
hair               306304 non-null object
allFaces           306304 non-null object
eliminatedFaces    306304 non-null object
dtypes: bool(1), object(9)
memory usage: 21.3+ MB


In [7]:
df = pd.DataFrame(all_rows, columns=['faceId', 'question', 'response', 'eliminated?',
                     'ethnicity', 'eyes', 'gender', 'hair', 'allFaces', 
                     'eliminatedFaces'])
df.head()

,faceId,question,response,eliminated?,ethnicity,eyes,gender,hair,allFaces,eliminatedFaces
0,IMG_9891,Is the person's hair long?,No,True,white,brown,female,blonde,"IMG_9891,IMG_0153,IMG_0446,IMG_9484,IMG_9846,I...","IMG_9891,IMG_0153,IMG_9846,IMG_8502,IMG_8198,I..."
1,IMG_0153,Is the person's hair long?,No,True,white,brown,female,brown,"IMG_9891,IMG_0153,IMG_0446,IMG_9484,IMG_9846,I...","IMG_9891,IMG_0153,IMG_9846,IMG_8502,IMG_8198,I..."
2,IMG_0446,Is the person's hair long?,No,False,white,brown,female,brown,"IMG_9891,IMG_0153,IMG_0446,IMG_9484,IMG_9846,I...","IMG_9891,IMG_0153,IMG_9846,IMG_8502,IMG_8198,I..."
3,IMG_9484,Is the person's hair long?,No,False,white,Blue,female,Blonde,"IMG_9891,IMG_0153,IMG_0446,IMG_9484,IMG_9846,I...","IMG_9891,IMG_0153,IMG_9846,IMG_8502,IMG_8198,I..."
4,IMG_9846,Is the person's hair long?,No,True,white,brown,male,brown,"IMG_9891,IMG_0153,IMG_0446,IMG_9484,IMG_9846,I...","IMG_9891,IMG_0153,IMG_9846,IMG_8502,IMG_8198,I..."


In [4]:
import re
def process_hair(): 
    hair = df["hair"]
    clean = []
    for line in hair:
        line = line.lower().strip()
        line = re.sub(r'\bbla\S*\b', "black", line)
        line = re.sub(r'\bbr\S*\b', "brown", line)
        if "blond" in line: 
            line = line.replace(line, "blond")
        if line == "dark": 
            line = line.replace(line, "black")
        if "medium brown" in line: 
            line = line.replace(line, "brown")
        if "grey brown" in line: 
            line = line.replace(line, "blond")
        clean.append(line)
    return clean


In [5]:
import re
def process_eyes(): 
    eyes = df["eyes"]
    clean = []
    for line in eyes:
        line = line.lower().strip()
        line = re.sub(r'\b\S*\wen\b', "green", line)
        if line == "grey": 
            line = line.replace(line, "black")
        if line == "black": 
             line = line.replace(line, "dark brown")
        clean.append(line)
    return clean

In [1]:
word_list = process_hair()
check = {}
for line in word_list: 
    count = check.get(line, 0)
    count += 1
    check[line] = count

        
print(check)

NameError: name 'process_hair' is not defined

In [9]:
#creating the target value: True Answer
def getTrueAnswer(): 
    question_answer = []
    index = 0
    for index in range(0, len(df["response"])):
        if df["eliminated?"][index] == True and df["response"][index] == "No":
            question_answer.append("yes")
        if df["eliminated?"][index] == False and df["response"][index] == "No":
            question_answer.append("no")
        if df["eliminated?"][index] == False and df["response"][index] == "Yes":
            question_answer.append("yes")    
        if df["eliminated?"][index] == True and df["response"][index] == "Yes":
            question_answer.append("no")
        index += 1
    return question_answer

In [10]:
hair = process_hair()
eyes = process_eyes()
true_answer = getTrueAnswer()

In [13]:
del df['hair']
del df['eyes']
df1 = pd.DataFrame({'hair': hair})
df2 = pd.DataFrame({'eyes': eyes})
df3 = pd.DataFrame({'trueAnswer': true_answer})
dfs = [df1, df2, df3]
dfs = pd.concat(dfs, axis = 1)

In [14]:
#columns=['faceId', 'question', 'trueAnswer' 'response', 'eliminated?','ethnicity', 'eyes', 'gender', 'hair', 'allFaces','eliminatedFaces'] 
data = pd.concat([dfs, df], axis=1)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306304 entries, 0 to 306303
Data columns (total 11 columns):
hair               306304 non-null object
eyes               306304 non-null object
trueAnswer         306304 non-null object
faceId             306304 non-null object
question           306304 non-null object
response           306304 non-null object
eliminated?        306304 non-null bool
ethnicity          297673 non-null object
gender             306304 non-null object
allFaces           306304 non-null object
eliminatedFaces    306304 non-null object
dtypes: bool(1), object(10)
memory usage: 23.7+ MB


In [15]:
outfile = open('processed-data.csv','w')
data.to_csv(outfile, index = False)
outfile.close()    
#file = data.to_csv('parsed_json/processed-data.csv')
# csv_file = open("parsed_json/processed-data.csv", "w")
# df.to_csv = (csv_file)
# csv_file.close()

In [16]:
import pandas as pd
data = pd.read_csv("processed-data.csv")
data

,hair,eyes,trueAnswer,faceId,question,response,eliminated?,ethnicity,gender,allFaces,eliminatedFaces
0,blond,brown,yes,IMG_9891,Is the person's hair long?,No,True,white,female,"IMG_9891,IMG_0153,IMG_0446,IMG_9484,IMG_9846,I...","IMG_9891,IMG_0153,IMG_9846,IMG_8502,IMG_8198,I..."
1,brown,brown,yes,IMG_0153,Is the person's hair long?,No,True,white,female,"IMG_9891,IMG_0153,IMG_0446,IMG_9484,IMG_9846,I...","IMG_9891,IMG_0153,IMG_9846,IMG_8502,IMG_8198,I..."
2,brown,brown,no,IMG_0446,Is the person's hair long?,No,False,white,female,"IMG_9891,IMG_0153,IMG_0446,IMG_9484,IMG_9846,I...","IMG_9891,IMG_0153,IMG_9846,IMG_8502,IMG_8198,I..."
3,blond,blue,no,IMG_9484,Is the person's hair long?,No,False,white,female,"IMG_9891,IMG_0153,IMG_0446,IMG_9484,IMG_9846,I...","IMG_9891,IMG_0153,IMG_9846,IMG_8502,IMG_8198,I..."
4,brown,brown,yes,IMG_9846,Is the person's hair long?,No,True,white,male,"IMG_9891,IMG_0153,IMG_0446,IMG_9484,IMG_9846,I...","IMG_9891,IMG_0153,IMG_9846,IMG_8502,IMG_8198,I..."
5,brown,brown,yes,IMG_8502,Is the person's hair long?,No,True,white,female,"IMG_9891,IMG_0153,IMG_0446,IMG_9484,IMG_9846,I...","IMG_9891,IMG_0153,IMG_9846,IMG_8502,IMG_8198,I..."
6,blond,hazel,yes,IMG_8198,Is the person's hair long?,No,True,white,female,"IMG_9891,IMG_0153,IMG_0446,IMG_9484,IMG_9846,I...","IMG_9891,IMG_0153,IMG_9846,IMG_8502,IMG_8198,I..."
7,blond,green,yes,IMG_7836,Is the person's hair long?,No,True,white,female,"IMG_9891,IMG_0153,IMG_0446,IMG_9484,IMG_9846,I...","IMG_9891,IMG_0153,IMG_9846,IMG_8502,IMG_8198,I..."
8,blond,brown,yes,IMG_8682,Is the person's hair long?,No,True,white,unsure,"IMG_9891,IMG_0153,IMG_0446,IMG_9484,IMG_9846,I...","IMG_9891,IMG_0153,IMG_9846,IMG_8502,IMG_8198,I..."
9,blond,brown,yes,IMG_0108,Is the person's hair long?,No,True,white,female,"IMG_9891,IMG_0153,IMG_0446,IMG_9484,IMG_9846,I...","IMG_9891,IMG_0153,IMG_9846,IMG_8502,IMG_8198,I..."
